## Halbautomatischer Dokumentenscanner 
Dieser halbautomatischer Dokumentenscanner ermöglicht den durch Mausklick auf dem Bild gefundenen Bereich direkt als Rechteckt darzustellen

In [2]:
import cv2
import numpy as np

point_lst = []
src_img = cv2.imread('poker_small.jpg')

COLOR = (255, 0, 255) # BGR -> pink

# Maus-Event
def mouse_handler(event, x, y, flags, param): # Diese Funktion wird aufgerufen, wenn ein Maus-Event auftritt
    if event == cv2.EVENT_LBUTTONDOWN: # linke Maustaste gedrückt
        point_lst.append((x, y)) # die Koordinaten des Maus-Events zur "point_lst" hinzufügen
        
    for point in point_lst:
        cv2.circle(src_img, point, 5, COLOR, cv2.FILLED)
        # einen Kreis um den Mittelpunkt(="point") zeichnen

    if len(point_lst) == 4:
        show_result() # die Funktion aufgerufen, um das Ergebnis anzuzeigen

    cv2.imshow('img', src_img)

def show_result():
    width, height = 210, 270
    src = np.float32(point_lst) # 4 Eingabepunkte, die durch Mausklick-Events erhalten wurden
    dst = np.array([[0, 0], [width, 0], [width, height], [0, height]], dtype = np.float32) # 4 Ausgabepunkte

    matrix = cv2.getPerspectiveTransform(src, dst) # die Transformationsmatrix erhalten, um den "scr"-Bereich in den "dst"-Bereich zu überfürhen
    result = cv2.warpPerspective(src_img, matrix, (width, height)) # die Transformationsmatrix anwenden, um das endgültige Ergebnisbild zu erhalten
    cv2.imshow('result', result)

# Neues Fenster für die Maus-Eventsverarbeitung
cv2.namedWindow('img')
cv2.setMouseCallback('img', mouse_handler)

cv2.imshow('img', src_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

Eine Gerade zur Überprüfung zeichnen, ob die eingegebenen Koordinaten korrekt sind

In [3]:
import cv2
import numpy as np

point_lst = []
src_img = cv2.imread('poker_small.jpg')

COLOR = (255, 0, 255)
THICKNESS = 3
drawing = False # bestimmt, ob eine Linie gezeicht werden soll
# Da die Koordinaten aus Mausklicks stammen müssen, ist es zunächst False, wird aber auf True gesetzt, wenn ein Klick-Event auftritt

def mouse_handler(event, x, y, flags, param):
    global drawing
    # Da der Wert von "drawing" geändert werden muss, wird das Schlüsselwort "global" verwendent, um auf die globale Variable "drawing" zuzugreifen

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True # mit dem Zeichnen der Linie beginnen
        point_lst.append((x, y))

    if drawing:
        prev_point = None # Startpunkt der Geraden
        for point in point_lst:
            cv2.circle(src_img, point, 5, COLOR, cv2.FILLED)
            
            if prev_point: # Gibt es einen "prev_point"? Dann eine Linie von "prev_point" bis zum aktuellen Punkt ("point") zeichnen
                cv2.line(src_img, prev_point, point, COLOR, THICKNESS, cv2.LINE_AA)
            prev_point = point # "prev_point" aktualisieren

    if len(point_lst) == 4:
        show_result()

    cv2.imshow('img', src_img)

def show_result():
    width, height = 210, 270
    src = np.float32(point_lst)
    dst = np.array([[0, 0], [width, 0], [width, height], [0, height]], dtype = np.float32)

    matrix = cv2.getPerspectiveTransform(src, dst)
    result = cv2.warpPerspective(src_img, matrix, (width, height))
    cv2.imshow('result', result)

cv2.namedWindow('img')
cv2.setMouseCallback('img', mouse_handler)

cv2.imshow('img', src_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

Echtzeit-Linie zeichnen  

In dem bisherigen Code wurden vier Punkte markiert, daher wurden nur drei Linien gezeichnet und anschließend direkt ein rechteckiges Bild erstellt.

In [4]:
import cv2
import numpy as np

point_lst = []
src_img = cv2.imread('poker_small.jpg')

COLOR = (255, 0, 255)
THICKNESS = 3
drawing = False

def mouse_handler(event, x, y, flags, param):
    global drawing
    dst_img = src_img.copy() # eine Kopie des "src_img" wird in "dst_img" gespeicher
    
    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        point_lst.append((x, y))

    if drawing:
        prev_point = None
        for point in point_lst:
            cv2.circle(dst_img, point, 5, COLOR, cv2.FILLED)
            
            if prev_point:
                cv2.line(dst_img, prev_point, point, COLOR, THICKNESS, cv2.LINE_AA)
            prev_point = point
            
        # In der for-Scheife werden nur Linien basierend auf den zuvor markierten vier Punkten gezeichnet
        
        next_point = (x, y) # Die Koordinaten (x, y) werden aus der "mouse_handler(x, y)-Funktion übernommen
        # Da keine separate Bedingung if event == cv2.EVENT_MOUSEMOVE:.. definiert wurde,
        # wird der "EVENT_LBUTTONDOWN"-Eventshandler ständig ausgeführt, während sich die Maus bewegt.
        # Daher werden die (x, y)-Koordinaten kontinuierlich aktualisiert

        if len(point_lst) == 4:
            show_result()
            next_point = point_lst[0] # Der erste Klickpunkt wird gespeichert
        cv2.line(dst_img, prev_point, next_point, COLOR, THICKNESS, cv2.LINE_AA)

    cv2.imshow('img', dst_img)

def show_result():
    width, height = 210, 270
    src = np.float32(point_lst)
    dst = np.array([[0, 0], [width, 0], [width, height], [0, height]], dtype = np.float32)

    matrix = cv2.getPerspectiveTransform(src, dst)
    result = cv2.warpPerspective(src_img, matrix, (width, height))
    cv2.imshow('result', result)

cv2.namedWindow('img')
cv2.setMouseCallback('img', mouse_handler)

cv2.imshow('img', src_img)

cv2.waitKey(0)
cv2.destroyAllWindows()